# Evaluating GPT-4o's Performance in Public Opinion Analysis
### Assessing its Ability in Determining Specific Brand Attributes from Social Media Images

### 1. Preparation
#### 1.1 Importing Required Packages

In [ ]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm
from openai import OpenAI
from google.colab import drive
import os
import openai

#### 1.2 Setting Up the OpenAI API Key

In [ ]:
client = OpenAI(api_key='')

#### 1.3 Defining the Prompt for Query

In [ ]:
label_attribution_prompt_text = """
Analyze the provided image, sourced from social media and related to a specific brand, to conduct a detailed public opinion analysis. Evaluate the image's content to determine whether it aligns with any of the following brand attributes. For each attribute, provide a label based on the visual and contextual cues observed in the image:

1. Glamorous / Drab: Assess the level of sophistication, elegance, or appeal present in the image.
2. Rugged / Gentle: Determine if the image conveys strength, toughness, or durability, versus softness, calmness, or subtlety.
3. Healthy / Unhealthy: Evaluate whether the image promotes or symbolizes health, wellness, and vitality, or if it indicates a lack of these qualities.
4. Fun / Dull: Assess whether the image conveys excitement, entertainment, or joy, compared to being uninteresting or lacking vibrancy.

Ensure your analysis considers the cultural and social context of the image, as well as its visual elements, and provide clear and concise labels for each attribute. Justify your labels with a brief explanation when possible.

When you answer, please only mention the label you choose. Do not include any other options. For example, if you choose "drab", do not mention "glamorous" at all in your answer.
"""

### 2. Rugged vs. Gentle Classification Study

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/CV_task/rugged_dataset.csv'
data = pd.read_csv(file_path)

# data = data.head(10) # test if our code works fine

true_labels = []
predicted_labels = []  # 1 for rugged, 0 for gentle
rugged_probs = []

group_results = {"top50": [], "middle50": [], "bottom50": []}
consumer_results = {}

for _, row in tqdm(data.iterrows(), desc="Processing images", total=len(data)):
    image_url = row['image_url']
    true_label = 1 if row['rugged'].strip().lower() == 'yes' else 0
    group = row['top_middle_bottom']  # Group classification (top50, middle50, bottom50)
    consumer = row['consumer_official']  # Consumer or official image

    if consumer not in consumer_results:
        consumer_results[consumer] = {"true_labels": [], "predicted_labels": []}

    # Get the image from the URL
    try:
        response = requests.get(image_url)
        # img = Image.open(BytesIO(response.content))

        print(f"Processing image from URL: {image_url}")
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()

        # Send the image URL to the GPT model
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": label_attribution_prompt_text,
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url},
                        },
                    ],
                }
            ],
        )
        result = response.choices[0].message.content.lower()

        # Determine predicted label (1 if rugged, 0 otherwise)
        predicted_label = 1 if "rugged" in result else 0
        predicted_labels.append(predicted_label)
        rugged_probs.append(1.0 if "rugged" in result else 0.0)
        true_labels.append(true_label)

        group_results[group].append((true_label, predicted_label))

        consumer_results[consumer]["true_labels"].append(true_label)
        consumer_results[consumer]["predicted_labels"].append(predicted_label)

        print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    except Exception as e:
        print(f"Failed to process image from URL: {image_url}. Error: {e}")

Processing images:   0%|                                | 0/150 [00:00<?, ?it/s]

Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/rugged/bottom50/1205441965593894330_1667356190.jpg


Processing images:   1%|▏                       | 1/150 [00:03<09:37,  3.88s/it]

True Label: 0, Predicted Label: 0
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/rugged/middle50/1242416075848572373_42698094.jpg


Processing images:   1%|▎                       | 2/150 [00:07<09:42,  3.94s/it]

...

True Label: 0, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/rugged/top50/12142151_1636383456627988_1700666281_n.jpg


Processing images:  99%|█████████████████████▊| 149/150 [12:11<00:04,  4.99s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/rugged/top50/13721276_1082949678451875_1748287275_n.jpg


Processing images: 100%|██████████████████████| 150/150 [12:16<00:00,  4.91s/it]

True Label: 1, Predicted Label: 0


In [ ]:
results_list = []

# Calculate Overall Accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)
results_list.append(["Overall Accuracy", overall_accuracy])
print(f"Overall Accuracy: {overall_accuracy}")

# Calculate Overall AUC
if len(set(true_labels)) > 1:
    overall_auc = roc_auc_score(true_labels, rugged_probs)
    results_list.append(["Overall AUC", overall_auc])
    print(f"Overall AUC: {overall_auc}")
else:
    print("AUC cannot be calculated as there is only one class in true labels.")

# Group-wise accuracy
for group, values in group_results.items():
    if values:  # Ensure the group has data
        group_true_labels, group_predicted_labels = zip(*values)
        group_accuracy = accuracy_score(group_true_labels, group_predicted_labels)
        results_list.append([f"Accuracy for group {group}", group_accuracy])
        print(f"Accuracy for {group}: {group_accuracy}")
    else:
        print(f"No data for group {group}.")

# Consumer/official accuracy
for consumer, results_dict in consumer_results.items():
    if results_dict["true_labels"]:  # Ensure the consumer has data
        consumer_accuracy = accuracy_score(results_dict["true_labels"], results_dict["predicted_labels"])
        results_list.append([f"Accuracy for consumer {consumer}", consumer_accuracy])
        print(f"Accuracy for consumer {consumer}: {consumer_accuracy}")
    else:
        print(f"No data for consumer {consumer}.")

df_results = pd.DataFrame(results_list, columns=["Metric", "Value"])
df_results.to_csv('test_results_rugged.csv', index=False)

Overall Accuracy: 0.9194630872483222
Overall AUC: 0.9240990171095741
Accuracy for top50: 0.96
Accuracy for middle50: 0.8
Accuracy for bottom50: 1.0
Accuracy for consumer user: 0.9264705882352942
Accuracy for consumer official: 0.9135802469135802


### 3. Glamorous / Drab Classification Study

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/CV_task/glamorous_dataset.csv'
data = pd.read_csv(file_path)

# data = data.head(10) # test if code works fine

true_labels = []
predicted_labels = []  # 1 for glamorous, 0 for drab
glamorous_probs = []

group_results = {"top50": [], "middle50": [], "bottom50": []}
consumer_results = {}

for _, row in tqdm(data.iterrows(), desc="Processing images", total=len(data)):
    image_url = row['image_url']
    true_label = 1 if row['glamorous'].strip().lower() == 'yes' else 0
    group = row['top_middle_bottom']  # Group classification (top50, middle50, bottom50)
    consumer = row['consumer_official']  # Consumer/official information

    # Initialize consumer group in the dictionary
    if consumer not in consumer_results:
        consumer_results[consumer] = {"true_labels": [], "predicted_labels": []}

    # Get the image from the URL
    try:
        response = requests.get(image_url)
        # img = Image.open(BytesIO(response.content))

        print(f"Processing image from URL: {image_url}")
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()

        # Send the image URL to the GPT model
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": label_attribution_prompt_text,
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url},
                        },
                    ],
                }
            ],
        )
        result = response.choices[0].message.content.lower()

        # Determine predicted label (1 if glamorous, 0 otherwise)
        predicted_label = 1 if "glamorous" in result else 0
        predicted_labels.append(predicted_label)
        glamorous_probs.append(1.0 if "glamorous" in result else 0.0)
        true_labels.append(true_label)

        group_results[group].append((true_label, predicted_label))

        consumer_results[consumer]["true_labels"].append(true_label)
        consumer_results[consumer]["predicted_labels"].append(predicted_label)

        print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    except Exception as e:
        print(f"Failed to process image from URL: {image_url}. Error: {e}")

Processing images:   0%|                                | 0/150 [00:00<?, ?it/s]

Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/glamorous/top50/12599203_1708675169344248_1887841860_n.jpg


Processing images:   1%|▏                       | 1/150 [00:12<31:08, 12.54s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/glamorous/middle50/1251832648518440479_2302182916.jpg


Processing images:   1%|▎                       | 2/150 [00:17<19:59,  8.10s/it]

...

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/glamorous/top50/11190220_1146166628733779_440442844_n.jpg


Processing images:  99%|█████████████████████▊| 149/150 [16:52<00:08,  8.64s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/glamorous/bottom50/1180517812483390714_1077271534.jpg


Processing images: 100%|██████████████████████| 150/150 [16:57<00:00,  6.78s/it]

True Label: 0, Predicted Label: 0


In [ ]:
results_list = []

# Calculate Overall Accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)
results_list.append(["Overall Accuracy", overall_accuracy])
print(f"Overall Accuracy: {overall_accuracy}")

# Calculate Overall AUC
if len(set(true_labels)) > 1:
    overall_auc = roc_auc_score(true_labels, glamorous_probs)
    results_list.append(["Overall AUC", overall_auc])
    print(f"Overall AUC: {overall_auc}")
else:
    print("AUC cannot be calculated as there is only one class in true labels.")

# Group-wise accuracy
for group, values in group_results.items():
    if values:  # Ensure the group has data
        group_true_labels, group_predicted_labels = zip(*values)
        group_accuracy = accuracy_score(group_true_labels, group_predicted_labels)
        results_list.append([f"Accuracy for group {group}", group_accuracy])
        print(f"Accuracy for {group}: {group_accuracy}")
    else:
        print(f"No data for group {group}.")

# Consumer/official accuracy
for consumer, results_dict in consumer_results.items():
    if results_dict["true_labels"]:  # Ensure the consumer has data
        consumer_accuracy = accuracy_score(results_dict["true_labels"], results_dict["predicted_labels"])
        results_list.append([f"Accuracy for consumer {consumer}", consumer_accuracy])
        print(f"Accuracy for consumer {consumer}: {consumer_accuracy}")
    else:
        print(f"No data for consumer {consumer}.")

df_results = pd.DataFrame(results_list, columns=["Metric", "Value"])
df_results.to_csv('test_results_glamorous.csv', index=False)


Overall Accuracy: 0.8053691275167785
Overall AUC: 0.8352272727272727
Accuracy for top50: 0.96
Accuracy for middle50: 0.673469387755102
Accuracy for bottom50: 0.78
Accuracy for consumer official: 0.8260869565217391
Accuracy for consumer user: 0.7875


### 4. Healthy / Unhealthy Prediction Study

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/CV_task/healthy_dataset.csv'
data = pd.read_csv(file_path)

# data = data.head(10) # test if code works fine

true_labels = []
predicted_labels = []  # 1 for healthy, 0 for unhealthy
healthy_probs = []

group_results = {"top50": [], "middle50": [], "bottom50": []}
consumer_results = {}

for _, row in tqdm(data.iterrows(), desc="Processing images", total=len(data)):
    image_url = row['image_url']
    true_label = 1 if row['healthy'].strip().lower() == 'yes' else 0
    group = row['top_middle_bottom']  # Group classification (top50, middle50, bottom50)
    consumer = row['consumer_official']  # Consumer official information

    # Initialize consumer group in the dictionary
    if consumer not in consumer_results:
        consumer_results[consumer] = {"true_labels": [], "predicted_labels": []}

    # Get the image from the URL
    try:
        response = requests.get(image_url)
        # img = Image.open(BytesIO(response.content))

        print(f"Processing image from URL: {image_url}")
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()

        # Send the image URL to the GPT model
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": label_attribution_prompt_text,
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url},
                        },
                    ],
                }
            ],
        )
        result = response.choices[0].message.content.lower()

        # Determine predicted label:
        # 1 if the result is "healthy", 0 if the result is "unhealthy"
        if "unhealthy" in result:
            predicted_label = 0
        elif "healthy" in result:
            predicted_label = 1
        else:
            predicted_label = 0  # Default to "unhealthy" if neither is explicitly mentioned

        predicted_labels.append(predicted_label)
        healthy_probs.append(1.0 if predicted_label == 1 else 0.0)  # Confidence: 1 for healthy, 0 for unhealthy
        true_labels.append(true_label)

        group_results[group].append((true_label, predicted_label))

        consumer_results[consumer]["true_labels"].append(true_label)
        consumer_results[consumer]["predicted_labels"].append(predicted_label)

        print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    except Exception as e:
        print(f"Failed to process image from URL: {image_url}. Error: {e}")


Processing images:   0%|                                | 0/150 [00:00<?, ?it/s]

Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/healthy/top50/1122499725393951973_588666310.jpg


Processing images:   1%|▏                       | 1/150 [00:09<23:03,  9.28s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/healthy/top50/1011886447341196298_39819980.jpg


Processing images:   1%|▎                       | 2/150 [00:13<15:38,  6.34s/it]

...

True Label: 0, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/healthy/bottom50/11098217_659419300830310_1255803697_n.jpg


Processing images:  99%|█████████████████████▊| 149/150 [13:24<00:05,  5.15s/it]

True Label: 0, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/healthy/bottom50/1219628370709933950_226337522.jpg


Processing images: 100%|██████████████████████| 150/150 [13:28<00:00,  5.39s/it]

True Label: 0, Predicted Label: 0


In [ ]:
results_list = []

# Calculate Overall Accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)
results_list.append(["Overall Accuracy", overall_accuracy])
print(f"Overall Accuracy: {overall_accuracy}")

# Calculate Overall AUC
if len(set(true_labels)) > 1:
    overall_auc = roc_auc_score(true_labels, healthy_probs)
    results_list.append(["Overall AUC", overall_auc])
    print(f"Overall AUC: {overall_auc}")
else:
    print("AUC cannot be calculated as there is only one class in true labels.")

# Group-wise accuracy
for group, values in group_results.items():
    if values:  # Ensure the group has data
        group_true_labels, group_predicted_labels = zip(*values)
        group_accuracy = accuracy_score(group_true_labels, group_predicted_labels)
        results_list.append([f"Accuracy for group {group}", group_accuracy])
        print(f"Accuracy for {group}: {group_accuracy}")
    else:
        print(f"No data for group {group}.")

# Consumer-wise accuracy
for consumer, results_dict in consumer_results.items():
    if results_dict["true_labels"]:  # Ensure the consumer has data
        consumer_accuracy = accuracy_score(results_dict["true_labels"], results_dict["predicted_labels"])
        results_list.append([f"Accuracy for consumer {consumer}", consumer_accuracy])
        print(f"Accuracy for consumer {consumer}: {consumer_accuracy}")
    else:
        print(f"No data for consumer {consumer}.")

df_results = pd.DataFrame(results_list, columns=["Metric", "Value"])
df_results.to_csv('test_results_healthy.csv', index=False)


Overall Accuracy: 0.5302013422818792
Overall AUC: 0.6287515006002401
Accuracy for top50: 0.8571428571428571
Accuracy for middle50: 0.5
Accuracy for bottom50: 0.24
Accuracy for consumer user: 0.6746987951807228
Accuracy for consumer official: 0.3484848484848485


### 5. Fun / Dull Classification Study

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Colab Notebooks/CV_task/fun_dataset.csv'
data = pd.read_csv(file_path)

# data = data.head(10) # test if code works fine

true_labels = []
predicted_labels = []  # 1 for fun, 0 for dull
fun_probs = []

group_results = {"top50": [], "middle50": [], "bottom50": []}
consumer_results = {}

for _, row in tqdm(data.iterrows(), desc="Processing images", total=len(data)):
    image_url = row['image_url']
    true_label = 1 if row['fun'].strip().lower() == 'yes' else 0
    group = row['top_middle_bottom']  # Group classification (top50, middle50, bottom50)
    consumer = row['consumer_official']  # Consumer/98official information

    # Initialize consumer group in the dictionary
    if consumer not in consumer_results:
        consumer_results[consumer] = {"true_labels": [], "predicted_labels": []}

    # Get the image from the URL
    try:
        response = requests.get(image_url)
        # img = Image.open(BytesIO(response.content))

        print(f"Processing image from URL: {image_url}")
        # plt.imshow(img)
        # plt.axis('off')
        # plt.show()

        # Send the image URL to the GPT model
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": label_attribution_prompt_text,
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_url},
                        },
                    ],
                }
            ],
        )
        result = response.choices[0].message.content.lower()

        # Determine predicted label (1 if fun, 0 otherwise)
        predicted_label = 1 if "fun" in result else 0
        predicted_labels.append(predicted_label)
        fun_probs.append(1.0 if "fun" in result else 0.0)
        true_labels.append(true_label)

        group_results[group].append((true_label, predicted_label))

        consumer_results[consumer]["true_labels"].append(true_label)
        consumer_results[consumer]["predicted_labels"].append(predicted_label)

        print(f"True Label: {true_label}, Predicted Label: {predicted_label}")
    except Exception as e:
        print(f"Failed to process image from URL: {image_url}. Error: {e}")


Processing images:   0%|                                | 0/150 [00:00<?, ?it/s]

Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/fun/middle50/1251786557016454481_248976228.jpg


Processing images:   1%|▏                       | 1/150 [00:05<12:27,  5.02s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/fun/top50/10576151_797937836894059_1106431855_n.jpg


Processing images:   1%|▎                       | 2/150 [00:09<10:56,  4.44s/it]

...

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/fun/middle50/11176575_1438773159754722_928027814_n.jpg


Processing images:  99%|█████████████████████▊| 149/150 [13:17<00:05,  5.19s/it]

True Label: 1, Predicted Label: 1
Processing image from URL: https://brandperception.s3.amazonaws.com/imagenet_eval_top50_middle50_bottom50/fun/bottom50/1239236415910053139_1834310768.jpg


Processing images: 100%|██████████████████████| 150/150 [13:22<00:00,  5.35s/it]

True Label: 0, Predicted Label: 0


In [ ]:
results_list = []

# Calculate Overall Accuracy
overall_accuracy = accuracy_score(true_labels, predicted_labels)
results_list.append(["Overall Accuracy", overall_accuracy])
print(f"Overall Accuracy: {overall_accuracy}")

# Calculate Overall AUC
if len(set(true_labels)) > 1:
    overall_auc = roc_auc_score(true_labels, fun_probs)
    results_list.append(["Overall AUC", overall_auc])
    print(f"Overall AUC: {overall_auc}")
else:
    print("AUC cannot be calculated as there is only one class in true labels.")

# Group-wise accuracy
for group, values in group_results.items():
    if values:  # Ensure the group has data
        group_true_labels, group_predicted_labels = zip(*values)
        group_accuracy = accuracy_score(group_true_labels, group_predicted_labels)
        results_list.append([f"Accuracy for group {group}", group_accuracy])
        print(f"Accuracy for {group}: {group_accuracy}")
    else:
        print(f"No data for group {group}.")

# Consumer-wise accuracy
for consumer, results_dict in consumer_results.items():
    if results_dict["true_labels"]:  # Ensure the consumer has data
        consumer_accuracy = accuracy_score(results_dict["true_labels"], results_dict["predicted_labels"])
        results_list.append([f"Accuracy for consumer {consumer}", consumer_accuracy])
        print(f"Accuracy for consumer {consumer}: {consumer_accuracy}")
    else:
        print(f"No data for consumer {consumer}.")

df_results = pd.DataFrame(results_list, columns=["Metric", "Value"])
df_results.to_csv('test_results_fun.csv', index=False)


Overall Accuracy: 0.5503355704697986
Overall AUC: 0.6277777777777778
Accuracy for top50: 0.9387755102040817
Accuracy for middle50: 0.4
Accuracy for bottom50: 0.32
Accuracy for consumer user: 0.4594594594594595
Accuracy for consumer official: 0.64
